Josh Barback  
`barback@fas.harvard.edu`  
Onnela Lab, Harvard T. H. Chan School of Public Health  

# Working with Beiwe study configurations

This notebook provides an overview of some features of the `beiwetools.configread` subpackage.

This module provides classes for representing Beiwe study settings that are stored in JSON configuration files.  These examples include code for the following tasks:

1. Read a Beiwe configuration file,
2. Query specific configuration settings,
3. Generate human-readable documentation,
4. Inspect a tracking survey question,
5. Assign names to study objects.

### 1. Read a Beiwe configuration file

In [1]:
import os
from beiwetools.configread import BeiweConfig

In [2]:
# Sample configuration files are located in examples/configuration_files.
examples_directory = os.getcwd() # change as needed
configuration_directory = os.path.join(examples_directory, 'configuration_files')

# We'll be looking at the following configuration file:
configuration_filename = 'generic_study.json'
configuration_path = os.path.join(configuration_directory, configuration_filename)

# Choose a directory for test ouput:
test_directory = os.path.join(examples_directory, 'test') # change as needed

In [3]:
# Read the configuration file into a BeiweConfig object:
config = BeiweConfig(configuration_path)

In [4]:
# A deserialized version of the original JSON file is retained as ordered dictionaries and lists.
print(config.raw)

OrderedDict([('device_settings', OrderedDict([('accelerometer_on_duration_seconds', 10), ('proximity', True), ('bluetooth', False), ('texts', True), ('gps_off_duration_seconds', 1200), ('gyro_on_duration_seconds', 60), ('magnetometer_off_duration_seconds', 1200), ('seconds_before_auto_logout', 600), ('wifi_log_frequency_seconds', 300), ('accelerometer', True), ('gps_on_duration_seconds', 60), ('magnetometer_on_duration_seconds', 60), ('consent_form_text', 'I have read and understood the information about the study and all of my questions about the study have been answered by the study staff.'), ('upload_data_files_frequency_seconds', 3600), ('devicemotion_on_duration_seconds', 60), ('power_state', True), ('gps', True), ('magnetometer', False), ('bluetooth_total_duration_seconds', 300), ('accelerometer_off_duration_seconds', 1250), ('about_page_text', 'The Beiwe application runs on your phone and helps researchers collect information about your behaviors. Beiwe may ask you to fill out s

In [5]:
# Undocumented settings or objects in the configuration file are logged as warnings.
# They are also stored in the 'warnings' attribute.  In this case, there should be no warnings:
print(config.warnings)

[]


### 2. Query specific configuration settings

In [6]:
# Study settings are organized into several ordered dictionaries.
# Due to differences across configuration file formats, it's normal that some settings are "Not found."

# For example, here are the settings for overall Beiwe app behavior:
config.settings.app

# Settings that affect all surveys:
# config.settings.survey

# Settings for passive data collection:
# config.settings.passive

# Settings for text displayed by the app:
# config.settings.display

# Undocumented / unknown settings:
# config.settings.other

OrderedDict([('allow_upload_over_cellular_data', False),
             ('create_new_data_files_frequency_seconds', 900),
             ('seconds_before_auto_logout', 600),
             ('upload_data_files_frequency_seconds', 3600),
             ('use_anonymized_hashing', 'Not found'),
             ('use_gps_fuzzing', 'Not found')])

In [7]:
# Documented settings are found in the file .../beiwetools/configread/study_settings.json
# Query a specific setting with its key:
config.settings.survey['voice_recording_max_time_length_seconds']

120

In [8]:
# To see all passive data settings for a particular sensor:
sensor = 'accelerometer'
for k in config.settings.passive.keys():
    if sensor in k: 
        print('%s: %s' % (k, config.settings.passive[k]))

accelerometer: True
accelerometer_off_duration_seconds: 1250
accelerometer_on_duration_seconds: 10


### 3. Generate human-readable documentation

In [9]:
# Human-readable summaries can be exported to text files.  
documentation_path = config.export(test_directory)
print(os.listdir(documentation_path))

# Documentation includes:
#     - human-readable summaries of settings and surveys,
#     - a log of any unknown settings or study objects,
#     - records of paths to JSON files used to instantiate this BeiweConfig object,
#     - pretty-printed copies of those JSON files.

['warnings.txt', 'settings', 'audio_surveys', 'tracking_surveys', 'records', 'overview.txt']


In [10]:
# Human-readable summaries can also be printed, for example:
config.summary.print() # configuration overview
config.settings.passive_summary.print() # settings for passive data collection


generic study

----------------------------------------------------------------------
Identifiers
----------------------------------------------------------------------
    Study Identifier (may not agree with backend):
        55d231c197013e3a1c9b8c30
    MongDB Extended JSON format: True
    Default names: True

----------------------------------------------------------------------
Number of Active Surveys
----------------------------------------------------------------------
    Audio: 1
    Tracking: 1

----------------------------------------------------------------------
Number of Deleted Surveys
----------------------------------------------------------------------
    Audio: 0
    Tracking: 0



----------------------------------------------------------------------
Passive Data Settings
----------------------------------------------------------------------
    accelerometer: True
    accelerometer_off_duration_seconds: 1250
    accelerometer_on_duration_seconds: 10
    bluetoo

In [11]:
# This study has one audio survey and one tracking survey.
# Their identifiers are:
audio_id = config.survey_ids['audio'][0]
tracking_id = config.survey_ids['tracking'][0]

# We can print summaries of these surveys:
for survey_id in [audio_id, tracking_id]:
    s = config.surveys[survey_id]
    s.summary.print()
    print('\n' + 70*'#' + '\n')


Survey 1

----------------------------------------------------------------------
Info
----------------------------------------------------------------------
    Identifier: 55db4c0597013e3fb50376a7
    Type: audio_survey
    Deleted: Not found

----------------------------------------------------------------------
Settings
----------------------------------------------------------------------
    audio_survey_type: compressed
    bit_rate: 64000
    trigger_on_first_download: False
    sample_rate: Not found

----------------------------------------------------------------------
Timings
----------------------------------------------------------------------
    Sun: 10:30
    Mon: 10:30
    Tue: 10:30
    Wed: 10:30
    Thu: 10:30
    Fri: 10:30
    Sat: 10:30

----------------------------------------------------------------------
Prompt
----------------------------------------------------------------------
    This is an Audio Survey.  Please read the following prompt: The
    functio

### 4. Inspect a tracking survey question

In [12]:
# First look up the question identifier:
question_id = config.identifier_lookup['Survey 2 - Question 3']

# Then get the corresponding TrackingQuestion object:
s2q3 = config.surveys[tracking_id].questions[question_id]

# And print a summary of this question:
s2q3.summary.print()


Survey 2 - Question 3

----------------------------------------------------------------------
Identifier
----------------------------------------------------------------------
    04e4c452-c7bd-4105-e1ce-389cee4f5d63

----------------------------------------------------------------------
Type
----------------------------------------------------------------------
    radio_button

----------------------------------------------------------------------
Logic
----------------------------------------------------------------------
    Does not use branching logic.

----------------------------------------------------------------------
Text
----------------------------------------------------------------------
    How many people did you talk to today?

----------------------------------------------------------------------
Answers
----------------------------------------------------------------------
    0: None
    1: One or two
    2: Three to five
    3: Six or more



In [13]:
# Specific attributes can be queried directly:
print(s2q3.type)
print(s2q3.text)
print(s2q3.answers)
print(s2q3.scores)

radio_button
How many people did you talk to today?
['None', 'One or two', 'Three to five', 'Six or more']
[0, 1, 2, 3]


In [14]:
# Branching logic isn't implemented for this question.
# If it were, the logic configuration could be viewed with:
print(s2q3.logic)

None


### 5. Assign names to study objects

In [15]:
# Default survey and question names are assigned in the order they appear in the original JSON file.
# For convenience, it may be desirable to assign different names to surveys and questions.
# Here are the current name assignments:
config.name_assignments

OrderedDict([('55d231c197013e3a1c9b8c30', 'generic study'),
             ('55db4c0597013e3fb50376a7', 'Survey 1'),
             ('575f0ee81206f707453870f7', 'Survey 2'),
             ('6c4ae7d9-6a69-4a58-ce65-aeb660a83e2d', 'Survey 2 - Question 1'),
             ('c67d8bec-9ea3-45a6-ee31-fe00ee58660c', 'Survey 2 - Question 2'),
             ('04e4c452-c7bd-4105-e1ce-389cee4f5d63',
              'Survey 2 - Question 3')])

In [16]:
# New name assignments should be unique.
# To assign new names:
old_names = list(config.name_assignments.values())
new_names = ['Generic Study', 'Sample Audio Survey', 'Sample Tracking Survey', 
             'Checkbox Example', 'Slider Example', 'Radio Button Example']
new_assignments = dict(zip(old_names, new_names))
config.update_names(new_assignments)

# The names have been updated:
config.name_assignments

OrderedDict([('55d231c197013e3a1c9b8c30', 'Generic Study'),
             ('55db4c0597013e3fb50376a7', 'Sample Audio Survey'),
             ('575f0ee81206f707453870f7', 'Sample Tracking Survey'),
             ('6c4ae7d9-6a69-4a58-ce65-aeb660a83e2d', 'Checkbox Example'),
             ('c67d8bec-9ea3-45a6-ee31-fe00ee58660c', 'Slider Example'),
             ('04e4c452-c7bd-4105-e1ce-389cee4f5d63', 'Radio Button Example')])

In [17]:
# Object summaries will use the new names.  For example:
config.surveys[tracking_id].summary.print()


Sample Tracking Survey

----------------------------------------------------------------------
Info
----------------------------------------------------------------------
    Identifier: 575f0ee81206f707453870f7
    Type: tracking_survey
    Deleted: Not found

----------------------------------------------------------------------
Settings
----------------------------------------------------------------------
    randomize: False
    trigger_on_first_download: True
    randomize_with_memory: False
    number_of_random_questions: Not found

----------------------------------------------------------------------
Timings
----------------------------------------------------------------------
    Sun: 09:00
    Mon: 09:00
    Tue: 09:00
    Wed: 09:00
    Thu: 09:00
    Fri: 09:00
    Sat: 09:00

----------------------------------------------------------------------
Questions
----------------------------------------------------------------------

    Checkbox Example:
        Identifier: 6c

In [18]:
# After renaming study objects, export the configuration.
documentation_path = config.export(test_directory)

# In the future, to keep the same name assignments, load the study configuration from the exported documentation.
config_from_export = BeiweConfig(documentation_path)

# Verify that study objects match:
print(config_from_export == config)

# Check that name assignments are retained:
print(config_from_export.name_assignments == config.name_assignments)

True
True


In [19]:
# If desired, uncomment the following lines and delete the test output directory:
#import shutil
#shutil.rmtree(test_directory)